# <center>Projeto 1</center>
# <center>Fake News Detection</center>
___
Nesse notebook serão descritos todos os passos e métodos utilizados para a resolução do projeto.
___

# <center> Objetivos do projeto </center>
- Utilização de método para análise de textos
- Aprimoramento de métodos de Machine Learning
___

## Conteúdo
1. [Importação](#1)
2. [Análise Exploratória](#2)
3. [Pipeline Inicial](#3)
4. [Grid Search - Best Parameters](#4)
5. [Resultados Finais](#5)<br>

<a id="1"></a>
## 1. Importação

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

<a id="2"></a>
## 2. Análise Exploratória

### 2.1. Dados do projeto

O projeto foi proposto pela Data Flair Training como projeto de inicialização em conhecimentos de Machine Learning. O projeto e os demais encontram-se [neste link](https://data-flair.training/blogs/data-science-project-ideas/). <br>

In [ ]:
df = pd.read_csv("data/news.csv", sep=",", index_col=0)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

### 2.2. Dicionário das variáveis

- **title**: Título da Noticia
- **text**: Texto da Noticia
- **label**: Classificação da Noticia como sendo FAKE ou REAL

In [ ]:
df.isna().sum()

Dados não possuem valores faltantes nas colunas.

### 2.3. Tamanho dos textos

O método trabalhado nesse projeto será o **Tfidfvectorizer**, um algoritmo que transformar texto em uma representação significativa de números, usado assim, para o treinamento dos algoritmos de Machine Learning e suas previsões.

Assim, a checagem de tamanho dos textos será utilizado para evidenciar tendências de quantidade de caracteres e possivelmente textos vazios no conjunto de dados.

In [ ]:
length = []
[length.append(len(str(text))) for text in df['text']]
df['length'] = length
df.head()

In [ ]:
df.length.describe()

Os valores mínimos e máximos revelam alguns *outliers* que estudados.

In [ ]:
print('Número de artigos entre [0, 20]: ', len(df[df.length.between(0, 20)]))
print(df['text'][df.length.between(0, 20)])

print('Número de artigos entre [50000, max]: ', len(df[df.length.between(50000, max(df.length))]))
print(df['text'][df.length.between(50000, max(df.length))])

In [ ]:
df = df.drop(df['text'][df.length.between(0, 20) | df.length.between(50000, max(df.length))].index, axis=0)
df.length.describe()

No total, 44 registros foram retirados do conjunto de dados por serem considerados *outliers*.

### 2.4. Limpeza dos dados

Os textos presentes na base de dados possuem vários caracteres que não agregam nenhuma informação adicional para o modelo durante seu processo de aprendizado, já que serão reconhecidas apenas palavras.

Assim será realizado uma limpeza da coluna *df.text* removendo esses caracteres.

In [ ]:
df.text[1]

In [ ]:
df['text'] = df.text.apply(lambda text: ' '.join(re.findall('\w+',text)))

In [ ]:
df.text[1]

<a id="3"></a>
## 3. Pipeline Inicial

### 3.1. Preparação dos dados

A preparação dos dados será constituída acerca da separação dos dados de treinamento (80%) e de teste (20%) do conjunto de dados disponíveis na base.<br>

In [ ]:
X = df.loc[:, df.columns != "label"]
y = df.loc[:, df.columns == "label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X["text"], y, test_size=0.2, random_state=42)

### 3.2. Transformação dos dados

Para o método Tfidfvectorizer utilizado, alguns parâmetros são necessários, como:
- **analyzer**: Especifica o grau de granularidade que o texto deve ser subdivido durante a análise (palavra ou partes de palavras)
- **stop_words**: Especifica a lista de palavras que devem ser desconsideradas já que se presume não serem informativas na representação do conteúdo do texto, e assim podem ser removidas para evitar que sejam interpretadas como um sinal de previsão.<br>

In [ ]:
tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words= 'english')

Foram usados as recomendações obtidas na documentação do método presente em: [**sklearn.feature_extraction.text.TfidfVectorizer**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
tfidf_train = tfidfvectorizer.fit_transform(X_train)
tfidf_test = tfidfvectorizer.transform(X_test)

Aplicação do método no conjunto de dados de treinamento, e replicação do mesmo processo no conjunto de dados de teste.

### 3.3. PassiveAgressiveClassifier

Para o objetivo do projeto, a classificação das notícia como sendo Fake (Falsa) ou Real, através da análise de textos de noticias, trata-se de um problema de Natural Languague Processing (NLP) ou Linguaguem de Processamento Natural.
Para isso, o método utilizado será o PassiveAgressiveClassifier, muito recomendado para a prática de classificação e análise de textos. No método os seguintes parâmetros serão utilizados:
- **C**: Tamanho máximo do passo (regularização), sendo o padrão de 1.0.
- **loss**: A função de perda a ser utilizada (hinge, squared_hinge).
- **max_iter**: O número máximo de passagens sobre os dados de treinamento (também conhecido como épocas).

In [ ]:
# Modelo inicial para avaliação
model = PassiveAggressiveClassifier(C=0.5, loss="hinge", max_iter=5, random_state=42)
  
# Treinamento do modelo com o conjunto de dados de treinamento
model.fit(tfidf_train, y_train)
  
# Realização as predições no conjunto de dados de teste
test_pred = model.predict(tfidf_test)

O modelo inicial com parâmetros aleatórios será utilizado de compreensão se o modelo adaptasse ou não a base de dados. Por fim, os métodos de avaliação dos resultados do modelo serão a **Acurácia** (grau de acertividade do modelo) e a **Matrix de Confusão** para visualização do quantidade de previsões assertivas e incorretas em cada classe de saída.

In [ ]:
# Avaliação do modelo
print(f"Acurácia Dados de Teste: {accuracy_score(y_test, test_pred) * 100} %\n\n")  
  
print(f"Matrix de Confusão: \n\n{confusion_matrix(y_test, test_pred)}")

Para um cenário inicial aleatório, o modelo obteve ótimos resultados com acurácia de **93.48%** de previsões corretas. Porém, os parâmetros foram estipulados de forma aleátória, o que demonstra que possa existir um conjunto de parâmetros que melhore os resultados do modelo ainda mais.

<a id="4"></a>
## 4. Grid Search - Best Parameters

O Grid Search é uma técnica de ajuste que tenta calcular os valores ótimos dos parâmetros de um modelo de aprendizado afim de encontrar os melhores resultados da avaliação do modelo.

É um método exaustivo, e muitas vezes computacionalmente inviável por requerer um longo período de análise, ele propõe determinar o melhor conjunto de parâmetros do modelo durante o treinamento do modelo, selecionando o conjunto que obtiver melhores resultados.

In [ ]:
# Definição do conjunto de parâmetros e valores a serem testados
param_grid = {'C' : [0.003, 0.01, 0.03, 0.1], 'loss': ['hinge', 'squared_hinge'], 'max_iter': [5, 10, 30, 100, 300]}

# Inicialização e treinamento do método
grid_search = GridSearchCV(model, param_grid)
grid_search.fit(tfidf_train, y_train)

# Visualização do melhor conjunto de parâmetros
grid_search.best_params_

Agora, é possível re-executar o método com o conjunto de parâmetros encontrados pelo método Grid Search, e analisar se houve ou não melhoria nos resultados obtidos.

<a id="5"></a>
## 5. Resultados Finais

In [ ]:
model = PassiveAggressiveClassifier(C= 0.03, loss= 'hinge', max_iter= 100, random_state=42)
  
# Treinamento do modelo com o conjunto de dados de treinamento
model.fit(tfidf_train, y_train)
  
# Realização as predições no conjunto de dados de teste
test_pred = model.predict(tfidf_test)
  
# Avaliação do modelo
print(f"Acurácia Dados de Teste: {accuracy_score(y_test, test_pred) * 100} %\n\n")  
  
print(f"Matrix de Confusão: \n\n{confusion_matrix(y_test, test_pred)}")

Observa-se uma melhor performance do modelo em suas previsões utilizando o conjunto de parâmetros definidos pelo método Grid Search como sendo os melhores dentre as opções. Assim o modelo final é computado com uma acurácia de **93.72%**. Ademais, é demonstrado a Matrix de Confusão utilizando a biblioteca de plotagem gráfica.

In [ ]:
labels = ['Fake', 'Real']
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(y_test, test_pred), annot=True, cmap="YlGnBu", fmt=".1f", xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Class')
plt.ylabel('Original Class')
plt.show()